In [ ]:
import warnings
warnings.filterwarnings("ignore")
from tslearn.clustering import TimeSeriesKMeans
from sklearn import preprocessing
import numpy as np
import pandas as pd
from tslearn import clustering
import matplotlib.pylab as plt
from tslearn.clustering import KShape
import matplotlib as mpl
from sklearn import metrics
from tslearn.metrics import soft_dtw
from tslearn.metrics import dtw
import seaborn as sns
import sys
from timeit import default_timer as timer

In [ ]:
# In this case we will analyze mortgage data. Input file has monthly state level Unemployment rate and DQ Pct from 2007 to 2020
# -- we will group 6-month state level Unemployment rate into different clusters
# -- and then analyze the 60-day Delinquency rate of change within a specific cluster

In [ ]:
def read_input_data(path1, path2):
    xl = pd.ExcelFile(path1)
    dfMacro = xl.parse(xl.sheet_names[0])

    xl = pd.ExcelFile(path2)
    dfDQ = xl.parse(xl.sheet_names[0])
    dfDQ["Date"] = pd.to_datetime(dfDQ['Date'],format="%m/%d/%Y")
    return dfMacro,dfDQ

def generate_training_data(df, length, step, start_index, end_index, col):
    # Store the start index of each time series in the training data
    X_index = []
    bStart = True
    index = start_index
    fullIndex = start_index
    states = df.columns.to_list()
    states.remove("Date")
    
    while index < end_index:
        index += step
        insideStep = 0
        for state in states:
            X_index.append([index,state])
            series = df[state].iloc[index:index+length].values.reshape(1,-1)
            series = preprocessing.normalize(series)
            if bStart:
                X_train = series
                bStart = False
            else:
                # concat new time series to training data
                X_train = np.vstack((X_train,series))
        index += step
    X_index = np.asarray(X_index)
    return (X_train, X_index)

In [ ]:
# Read input UE and DQ data.
# Take 6-month UE as training data, so each time series of length 6
# We will normarlize each observation in L2 form, which means if each element were squared and summed, the total would equal 1
dataMacro,dataDQ = read_input_data("StateUE_noDakotas.xls","MortgageDelinquency.xlsx")
targetField = "DQ60Pct"
n_cluster = 10
length = 6
step = 5
start_index = 0
dqMonths = 3
end_index = len(dataMacro) - 100
X_train, X_index = generate_training_data(dataMacro, length, step, start_index, end_index, 'Close')

In [ ]:
# Run Sklearn TimeSeriesKMeans model with metric dtw
# Store predicted labels on labels_train
metric = 'dtw'
km = TimeSeriesKMeans(n_clusters=n_cluster,metric=metric)
labels_train = km.fit_predict(X_train)

In [ ]:
# Define a scoring function to measure the cluster performance
# introduce Silhouette coefficient 
# -- a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation)

# compute dtw similarity between two time series
def calculate_dist(X, Y):
    return dtw(X,Y)

# compute average dtw distance between a time series and a whole cluster
def calculate_mean_dist(X, arr, mode):
    nsample = arr.shape[0]
    total = 0
    if mode == 0: # within cluster
        for series in arr:
            if np.array_equal(X, series):
                continue
            dist = calculate_dist(X, series)
            total += dist
        return (total / (nsample - 1))
    else: # inter-cluster
        for series in arr:
            dist = calculate_dist(X, series)
            total += dist
        return (total / nsample)
    
# compute the average silhouette score of a specific cluster
'''
for a given cluster, loop through each time series in the cluster, calculate its silhouette coefficient
and then return the average as silhouette score for the entire cluster
coefficient calculated based on within-cluster distance and min inter-cluster distance
'''
def calculate_cluster_score(i_cluster, df, labels, n_cluster):
    X_cluster_in = df[labels == i_cluster]
    cluster_score = 0
    count = 0
    for obs in X_cluster_in:
        d_in = calculate_mean_dist(obs, X_cluster_in, 0)
        count += 1
        bFirst = True
        for x_cluster in range(n_cluster):
            if x_cluster == i_cluster:
                continue
            X_cluster_out = df[labels == x_cluster]
            d_out = calculate_mean_dist(obs, X_cluster_out, 1)
            if bFirst:
                d_out_min = d_out
                bFirst = False
            else:
                if d_out < d_out_min:
                    d_out_min = d_out
        d_score = (d_out_min - d_in) / (max(d_out_min, d_in))
        cluster_score += d_score
    return (i_cluster, (cluster_score / count))

In [ ]:
# Apply the score function
cluster_quality = []
for i in range(n_cluster):
    cluster_quality.append(calculate_cluster_score(i, X_train, labels_train, n_cluster))
cluster_quality = sorted(cluster_quality,reverse=True, key=lambda x: x[1])
cluster_quality

In [ ]:
# now we want to analyze the rate of change for Delinquency data
# for each time series in the cluster, we select end month T and then calculate the DQ60 rate of change at T+1 (1 month later)
def get_cluster_dq(dfMacro, dfDQ, index_arr, dqMonths, step):
    dq_arr = []
    for stateInd in index_arr:
        macroIndex = int(stateInd[0])
        try:
            r1 = dfDQ.loc[(dfDQ["State"] == stateInd[1]) & (dfDQ["Date"] == dfMacro.iloc[macroIndex + step]["Date"]),targetField].values[0]
            r2 = dfDQ.loc[(dfDQ["State"] == stateInd[1]) & (dfDQ["Date"] == dfMacro.iloc[macroIndex + step + 1]["Date"]),targetField].values[0]
            r = (r2 / r1 - 1) * 100
        except:
            print("data not found for ", stateInd[1], dfMacro.iloc[macroIndex + step]["Date"])
            r = 0
        dq_arr.append(r)
    dq_arr = np.asarray(dq_arr)
    return dq_arr

In [ ]:
# Plot Unemployment rate trend and DQ rate of change graphs for the three best clusters
fig, ax = plt.subplots(2, 3, figsize=(20,10))
fig.suptitle('Unemployment and DQ rate of change plot')
index = 0
for clusterNum in range(3):
    cluster = cluster_quality[clusterNum][0]
    X_cluster = X_train[labels_train == cluster]
    X_cluster_index = X_index[labels_train == cluster]
    dq_arr = get_cluster_dq(dataMacro, dataDQ, X_cluster_index, dqMonths, step)
    ax[0, index].plot(range(1,length+1),np.median(X_cluster, axis=0))
    ax[0, index].set_title('Unemployment rate')
    sort_return = np.sort(dq_arr)
    y = np.arange(1, len(sort_return)+1) / len(sort_return)
    med_val = np.median(sort_return)
    median = np.array([med_val for i in range(len(y))])
    ax[1, index].plot(sort_return, y, marker='.', linestyle='none')
    ax[1, index].plot(median, y, label='median')
    ax[1, index].set_title('ECDF plot for DQ rate of change')
    ax[1, index].set_xticks(np.arange(np.floor(np.min(sort_return)),np.ceil(np.max(sort_return)) + 1,step=3))
    ax[1, index].set_yticks(np.arange(0,1.1,step=0.1))
    ax[1, index].set_xlabel(targetField)
    ax[1, index].set_ylabel('ECDF')
    ax[1, index].legend()
    index += 1